In [141]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import json
import folium

# data paths
TASK_1_DATA_PATH = './data/lfsa_urgan_1_Data.csv'
TASK_2_DATA_PATH = './data/Arbeitslosenquoten_task_2_1.csv'
TASK_2_2_DATA_PATH = './data/Arbeitslosenquoten_task_2_2.csv'

# Task 1: European unemployment rate (Eurostat)

### Getting the data from [Eurostat](http://ec.europa.eu/eurostat/data/database)

Database by themes/Population and social conditions/Labour market (Labour)/Employment and Unemployment(Labour Force survey)(employ)/LFS series - detailed annual survey results (LFSA)/ Total unemployment - LFS series(lfsa_unemp)/  Unemployment rates by sex, age and nationality (%) (lfsa_urgan)



Age from 15 to 74

How we got the data:
- we used the data explorer, then 'Download' button, 'Change Selection' to keep only what we're interested in, then download data in CSV format

### Data importing and cleaning
First, we import the unemployement rates from the Eurostat website in a dataframe. Since we are looking for recent statistics, we only keep the data from last year (2016). We also discard all columns except the ones we need (Country and unemployment rate). A few countries also need to be renamed, so that the countries' names are consistent between the eurostat DataFrame and the topojson file. We also drop some unwanted rows, such as the average rate for countries from the European Union.

In [2]:
df = pd.read_csv(TASK_1_DATA_PATH)
df = df[df['TIME']==2016] # only keep data from last year
df = df[['GEO', 'Value']] # The only information we need is the country and the unemployment rate
df.rename(columns={'GEO': 'Country', 'Value' : 'Unemployment rate' }, inplace=True)
df.index = (range(len(df))) # re-index the dataframe

# drop unwanted rows, rename some countries to match with the topojson data
df = df.replace(to_replace='Former Yugoslav Republic of Macedonia, the', value='The former Yugoslav Republic of Macedonia')
df = df.replace(to_replace='Germany (until 1990 former territory of the FRG)', value='Germany')
df = df.drop(index=[0,1, 2, 3, 4, 5])

df.index = (range(len(df))) # re-index the dataframe
df.head()

,Country,Unemployment rate
0,Belgium,7.8
1,Bulgaria,7.6
2,Czech Republic,4.0
3,Denmark,6.2
4,Germany,4.1


Then, we import the topojson data:

In [3]:
print(pd.__version__)

0.21.0


In [4]:
state_geo_path = r'topojson/europe.topojson.json'
geo_json_data = json.load(open(state_geo_path))

Now, let's see if we are missing some data. To do this, we extract the list of countries from the topojson file and check if some of these countries are not in our DataFrame:

In [5]:
# First, extract the list of countries from the topojson
countries = [country['properties']['NAME'] for country in geo_json_data['objects']['europe']['geometries']]
# From this list, print each country that does not appear in the DataFrame
missing_countries = set(countries).difference(set(df.Country.values))
print("The data is missing for %d out of %d countries:\n" %(len((missing_countries)), len(countries)))
print('\n'.join(missing_countries))

The data is missing for 18 out of 51 countries:

Russia
Armenia
San Marino
Montenegro
Bosnia and Herzegovina
Republic of Moldova
Faroe Islands
Ukraine
Belarus
Monaco
Holy See (Vatican City)
Israel
Azerbaijan
Serbia
Albania
Liechtenstein
Andorra
Georgia


Let's ensure that each countries from the DataFrame has been matched to a country in the topojson:

In [6]:
assert(not set(df.Country.values).difference(set(countries)))

Because we don't have the unemployment rate of some countries in topojson file, we need to delete them such that our choropleth plot does not get spoiled.

In [7]:
index_to_keep = [country['properties']['NAME'] in df.Country.values 
                for country in geo_json_data['objects']['europe']['geometries']]
geo_json_data['objects']['europe']['geometries'] = np.array(geo_json_data['objects']['europe']['geometries'])[index_to_keep].tolist()

### Build the choropleth map

In [8]:
m = folium.Map([53, 15],  tiles='cartodbpositron', zoom_start=4)
#ignore_missing_countries = lambda x: {'fillOpacity':1,'fillColor':'YlOrRd' if x['properties']['NAME'] in df.Country.values else 'black'}
#folium.TopoJson(open(state_geo_path), 'objects.europe', style_function =ignore_missing_countries).add_to(m)
m.choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=df,
    columns=['Country', 'Unemployment rate'],
    key_on='feature.properties.NAME',
    topojson='objects.europe',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=1,
    legend_name='Unemployment Rate (%)'
)
m

In [9]:
from IPython.display import IFrame

IFrame('http://localhost:8888/notebooks/ADA_HW/AppliedDataAnalysis/Homeworks/03%20-%20Interactive%20Viz/europe_unemployment_rate.html', width=800, height=600)

# Task 2: Unemployment rate in Switzerland
this dataframe holds the information from september 2017

## Task 2.1

In [ ]:
def clean_amstat_dataframe(df):
    
    

In [99]:
df_ch = pd.read_csv(TASK_2_DATA_PATH)

# drop unwanted columns
cols_to_drop = [i for i in range(1, len(df_ch.columns), 2)]
cols_to_drop.append(20)
cols_to_drop.append(22)
df_ch.drop(df_ch.columns[cols_to_drop],axis=1, inplace=True)

#drop unwanted row
df_ch.drop(index=[0, len(df_ch)-1], axis=0, inplace=True)

# set cantons as index
df_ch.set_index('Kanton', drop=True, inplace=True)

# cast dataframe values as float
df_ch=df_ch.astype(float)

# compute the average unemployment rate for each canton
df_ch['unemployment rate'] = df_ch.mean(axis=1)
df_ch.head()

,Januar 2017,Februar 2017,März 2017,April 2017,Mai 2017,Juni 2017,Juli 2017,August 2017,September 2017,unemployment rate
Kanton,,,,,,,,,,
Zürich,3.9,3.9,3.8,3.6,3.5,3.4,3.4,3.4,3.3,3.577778
Bern,3.0,3.0,2.9,2.7,2.6,2.4,2.4,2.5,2.4,2.655556
Luzern,2.2,2.2,2.0,1.9,1.8,1.7,1.7,1.8,1.7,1.888889
Uri,1.5,1.4,1.3,1.2,1.0,0.8,0.7,0.6,0.6,1.011111
Schwyz,2.0,2.0,1.9,1.8,1.7,1.6,1.7,1.7,1.7,1.788889


In [122]:
topo_ch = r'topojson/ch-cantons.topojson.json'
topo_ch_data = json.load(open(topo_ch))

In [123]:
# First, extract the list of cantons from the topojson
cantons_topo = [canton['properties']['name'] for canton in topo_ch_data['objects']['cantons']['geometries']]
# From this list, print each canton that does not appear in the DataFrame
missing_cantons = set(cantons_topo).difference(set(df_ch.index.values))
print("Some names are not matching %d / %d" %(len(missing_cantons), len(cantons_topo)))
print('\n'.join(missing_cantons))

Some names are not matching 0 / 26



In [124]:
# replace a few names for the cantons whose names are not matching between topojson and dataframe
df_ch = df_ch.rename(index={'Genf':'Genève', 'Waadt':'Vaud', 'Neuenburg':'Neuchâtel', 'Graubünden':'Graubünden/Grigioni', 'Tessin':'Ticino', 'Bern':'Bern/Berne', 'Wallis':'Valais/Wallis','Freiburg':'Fribourg'})

# assert that every name is matching
assert(set(cantons_topo) == set(df_ch.index.values))

# we need a 'cantons' column to bind the data with folium
df_ch['cantons'] = df_ch.index

In [136]:
map_ch = folium.Map([46.9, 8.2],  tiles='cartodbpositron', zoom_start=8)
map_ch.choropleth(
    geo_data=topo_ch_data,
    name='choropleth',
    data=df_ch,
    columns=['cantons', 'unemployment rate'],
    key_on='feature.properties.name',
    topojson='objects.cantons',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=1,
    legend_name='Unemployment Rate (%)'
)
map_ch

## Task 2.2

In [164]:
df_ch2 = pd.read_csv(TASK_2_2_DATA_PATH)


# set cantons as index
df_ch2.set_index('Kanton', drop=True, inplace=True)

# drop unwanted columns
cols_to_drop = [i for i in range(len(df_ch2.columns)-6, len(df_ch2.columns))]
cols_to_drop.append(0)
df_ch2.drop(df_ch2.columns[cols_to_drop],axis=1, inplace=True)

# for each month, compute the percentage of unemployed people, excluding those that already have a job but are looking for a new one
for i in range(0, len(df_ch2.columns), 3):
    ratio = df_ch2.iloc[:,i]
    total_unemployed = df_ch2.iloc[:,i+1]
    total_looking = df_ch2.iloc[:,i+2]
    result = (ratio/total_unemployed)*(total_unemployed-total_looking)
    
    

'''
# drop unwanted columns
cols_to_drop = [i for i in range(1, len(df_ch2.columns), 2)]
cols_to_drop.append(20)
cols_to_drop.append(22)
df_ch2.drop(df_ch2.columns[cols_to_drop],axis=1, inplace=True)


#drop unwanted row
df_ch2.drop(index=[0, len(df_ch)-1], axis=0, inplace=True)

# cast dataframe values as float
df_ch2=df_ch2.astype(float)

# compute the average unemployment rate for each canton
df_ch2['unemployment rate'] = df_ch2.mean(axis=1)
df_ch2.head()
'''

df_ch2

,Januar 2017,Januar 2017.1,Januar 2017.2,Februar 2017,Februar 2017.1,Februar 2017.2,März 2017,März 2017.1,März 2017.2,April 2017,...,Juni 2017.2,Juli 2017,Juli 2017.1,Juli 2017.2,August 2017,August 2017.1,August 2017.2,September 2017,September 2017.1,September 2017.2
Kanton,,,,,,,,,,,,,,,,,,,,,
,Arbeitslosenquote,Registrierte Arbeitslose,Nicht arbeitslose Stellensuchende,Arbeitslosenquote,Registrierte Arbeitslose,Nicht arbeitslose Stellensuchende,Arbeitslosenquote,Registrierte Arbeitslose,Nicht arbeitslose Stellensuchende,Arbeitslosenquote,...,Nicht arbeitslose Stellensuchende,Arbeitslosenquote,Registrierte Arbeitslose,Nicht arbeitslose Stellensuchende,Arbeitslosenquote,Registrierte Arbeitslose,Nicht arbeitslose Stellensuchende,Arbeitslosenquote,Registrierte Arbeitslose,Nicht arbeitslose Stellensuchende
Zürich,3.9,32'387,6'953,3.9,31'619,7'129,3.8,30'841,7'210,3.6,...,7'209,3.4,27'992,7'120,3.4,27'514,7'028,3.3,27'225,6'931
Bern,3,16'954,4'632,3,16'738,4'629,2.9,16'035,4'854,2.7,...,4'640,2.4,13'633,4'809,2.5,13'829,4'564,2.4,13'658,4'727
Luzern,2.2,4'985,2'965,2.2,4'808,3'049,2,4'493,3'081,1.9,...,3'030,1.7,3'875,2'930,1.8,3'992,2'861,1.7,3'885,2'871
Uri,1.5,297,167,1.4,276,181,1.3,256,175,1.2,...,135,0.7,129,136,0.6,123,144,0.6,112,145
Schwyz,2,1'794,787,2,1'766,802,1.9,1'670,818,1.8,...,829,1.7,1'447,785,1.7,1'466,774,1.7,1'455,774
Obwalden,1,217,161,1.1,228,175,1,217,159,1,...,155,0.8,176,134,0.8,164,158,0.7,153,166
Nidwalden,1.3,305,230,1.3,306,245,1.2,297,252,1.1,...,197,1,241,193,1,247,178,1,248,188
Glarus,2.5,567,309,2.6,576,315,2.4,551,315,2.3,...,294,1.8,415,290,1.9,435,295,1.8,416,297
